this script includes: 

(1) turn the manual contours from control points to masks

(2) only keep the slices that have segmentation (LV + myo)

(3) n4 bias correction

(4) min-max normalization

(5) keep the voxel dimension uniform [1.25, 1.25, 2]

(6) make ROI image for LV and myocardium

In [1]:
import CMR_HFpEF_Analysis.Defaults as Defaults
import CMR_HFpEF_Analysis.functions_collection as ff
import CMR_HFpEF_Analysis.Image_utils as util

import os
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
import json
import SimpleITK as sitk
import pandas as pd
# import cv2
from scipy.ndimage import zoom


main_path = '/mnt/mount_zc_NAS/HFpEF/data/HFpEF_data/'

ModuleNotFoundError: No module named 'CMR_HFpEF_Analysis'

# main script (the product will be used in radiomics as the next step)

In [ ]:
# main script
case_list = ff.find_all_target_files(['ID_*'], os.path.join(main_path, 'contours'))

for i in range(0,len(case_list)):
    case = case_list[i]
    case_id = os.path.basename(case)
    print(i, case_id)

    save_folder = os.path.join(main_path, 'masks', case_id)
    if os.path.isfile(os.path.join(save_folder, 'mask_myo.nii.gz')) == 1:
        print('done, continue')
        continue

    # find the ED 
    edes_file = os.path.join(main_path, 'ED_ES', case_id,'ED_ES.txt')
    # prepare a blank txt:
    with open(edes_file, 'r') as file:
        line = file.readline().strip()
        numbers = line.split()
        ed = int(numbers[0])
    print(ed)
    # load the ED image
    img_file = os.path.join(main_path, 'raw_img', case_id, 'Org3D_frame'+str(ed) +'.nrrd')
    img_file = sitk.ReadImage(img_file)
    spacing = img_file.GetSpacing()
    print(spacing)
    img = np.rollaxis(sitk.GetArrayFromImage(img_file),0,3)

    # load contour points and turn into masks
    # contour folder
    contour_folder = os.path.join(main_path, 'contours',  case_id, 'manual_contours')
    endo_files = ff.sort_timeframe(ff.find_all_target_files(['Endo*'],contour_folder),2,'_')
    epi_files = ff.sort_timeframe(ff.find_all_target_files(['Epi*'],contour_folder),2,'_')
    assert len(endo_files) == len(epi_files)

    # convert to mask
    seg = np.zeros(img.shape)

    for file_i in range(0,len(endo_files)):
        slice_index = ff.find_timeframe(endo_files[file_i], 2, '_')
            
        endo_pts = []; epi_pts = []

        # get endo and epi contour points
        with open(endo_files[file_i], 'r') as f:
            endo_data = json.load(f)
            endo_data = endo_data['markups'][0]['controlPoints']
        for l in range(0,len(endo_data)):
            # if it's manual drawing from 3D slicer, need to convert from mm to pixel unit
            endo_pts.append([endo_data[l]['position'][0] / spacing[0], endo_data[l]['position'][1] / spacing[1]])

        with open(epi_files[file_i], 'r') as f:
            epi_data = json.load(f)
        epi_data = epi_data['markups'][0]['controlPoints']
        for l in range(0,len(epi_data)):
            # if it's manual drawing from 3D slicer, need to convert from mm to pixel unit
            epi_pts.append([epi_data[l]['position'][0] / spacing[0], epi_data[l]['position'][1] / spacing[1]])

        cts_dict = {"Endo":np.asarray(endo_pts).astype(int), "Epi": np.asarray(epi_pts).astype(int), "RV": None}

        seg[:,:,slice_index] = util.contourpts_to_mask(cts_dict, np.zeros([img.shape[0], img.shape[1]]),  ['LV', 'Myo'], [1,2], sample_rate = 1)
    
    # do some image processing
    # only keep the slices that have segmentation
    non_zero_slices = [z for z in range(seg.shape[2]) if np.sum(seg[:, :, z]) != 0]
    img_heart = img[:,:, non_zero_slices]
    seg_heart = seg[:,:,non_zero_slices]

    # n4 bias correction
    I = sitk.GetImageFromArray(img_heart)
    I = sitk.Cast(I, sitk.sitkFloat32)
    I = sitk.N4BiasFieldCorrection(I)
    img_bias_corrected = sitk.GetArrayFromImage(I)

    # make the min_max normalization into [0,255]
    img_norm = ((img_bias_corrected - np.min(img_bias_corrected)) / (np.max(img_bias_corrected) - np.min(img_bias_corrected))) * 255

    # make the voxel dimension uniform [1.25, 1.25, 2.0]
    new_voxel_dim = [1.25,1.25,2]

    zoom_factors = [original_dim / new_dim for new_dim, original_dim in zip(new_voxel_dim, spacing)]
    img_zoom = zoom(img_norm, zoom_factors, order = 3)  # Order=3 for cubic intenrpolation
    img_zoom = np.round(img_zoom)
    seg_zoom = zoom(seg_heart, zoom_factors, order = 0) # Order = 0 for nearest neighbour
    seg_zoom = np.round(seg_zoom)

    seg_lv_zoom = np.copy(seg_zoom); seg_lv_zoom[seg_lv_zoom != 1] = 0
    seg_myo_zoom = np.copy(seg_zoom); seg_myo_zoom[seg_myo_zoom != 2] = 0; seg_myo_zoom[seg_myo_zoom == 2] = 1

    ff.make_folder([save_folder])
    ff.save_itk(np.rollaxis(img_zoom,2,0), os.path.join(save_folder, 'img.nii.gz'), img_file, new_voxel_dim, np.eye(4))
    ff.save_itk(np.rollaxis(seg_zoom,2,0), os.path.join(save_folder, 'mask.nii.gz'), img_file, new_voxel_dim, np.eye(4))
    ff.save_itk(np.rollaxis(seg_lv_zoom,2,0), os.path.join(save_folder, 'mask_lv.nii.gz'), img_file, new_voxel_dim, np.eye(4))
    ff.save_itk(np.rollaxis(seg_myo_zoom,2,0), os.path.join(save_folder, 'mask_myo.nii.gz'), img_file, new_voxel_dim, np.eye(4))

    
    


just to process the data

In [6]:
# main script
case_list = ff.find_all_target_files(['ID_*'], os.path.join(main_path, 'unchecked/contours'))

for i in range(0,len(case_list)):
    case = case_list[i]
    case_id = os.path.basename(case)
    case_id_num = ff.ID_00XX_to_XX(case_id)
    print(case_id,case_id_num)

    save_img_folder = os.path.join(main_path, 'unchecked/nii_img', case_id)
    save_seg_folder = os.path.join(main_path, 'unchecked/nii_manual_seg', case_id)

    if os.path.isfile(os.path.join(main_path, 'nii_manual_seg',case_id, 'SAX_ED_seg.nii.gz' )) == 1:
        print('have checked segmentation, continue')
        continue

    # find the ED 
    full_list = pd.read_excel(os.path.join(main_path, 'Patient_list', 'full_list.xlsx'))
    ed = int(full_list.loc[full_list['OurID'] == case_id_num]['ED'])
    es = int(full_list.loc[full_list['OurID'] == case_id_num]['ES'])
    
    # load the ED image
    img_file = os.path.join(main_path, 'nrrd', 'need_' + case_id, 'Org3D_frame'+str(ed) +'.nrrd')
    img_file = sitk.ReadImage(img_file)
    img = sitk.GetArrayFromImage(img_file)
    spacing = img_file.GetSpacing()
    print(spacing)
    
    img = np.rollaxis(sitk.GetArrayFromImage(img_file),0,3)

    # load contour points and turn into masks
    # contour folder
    contour_folder = os.path.join(main_path, 'unchecked/contours',  case_id, 'manual_contours')
    endo_files = ff.sort_timeframe(ff.find_all_target_files(['Endo*'],contour_folder),2,'_')
    epi_files = ff.sort_timeframe(ff.find_all_target_files(['Epi*'],contour_folder),2,'_')
    assert len(endo_files) == len(epi_files)

    # convert to mask
    seg = np.zeros(img.shape)

    for file_i in range(0,len(endo_files)):
        slice_index = ff.find_timeframe(endo_files[file_i], 2, '_')
            
        endo_pts = []; epi_pts = []

        # get endo and epi contour points
        with open(endo_files[file_i], 'r') as f:
            endo_data = json.load(f)
            endo_data = endo_data['markups'][0]['controlPoints']
        for l in range(0,len(endo_data)):
            # if it's manual drawing from 3D slicer, need to convert from mm to pixel unit
            endo_pts.append([endo_data[l]['position'][0] / spacing[0], endo_data[l]['position'][1] / spacing[1]])

        with open(epi_files[file_i], 'r') as f:
            epi_data = json.load(f)
        epi_data = epi_data['markups'][0]['controlPoints']
        for l in range(0,len(epi_data)):
            # if it's manual drawing from 3D slicer, need to convert from mm to pixel unit
            epi_pts.append([epi_data[l]['position'][0] / spacing[0], epi_data[l]['position'][1] / spacing[1]])

        cts_dict = {"Endo":np.asarray(endo_pts).astype(int), "Epi": np.asarray(epi_pts).astype(int), "RV": None}

        seg[:,:,slice_index] = util.contourpts_to_mask(cts_dict, np.zeros([img.shape[0], img.shape[1]]),  ['LV', 'Myo'], [1,2], sample_rate = 1)

    ff.make_folder([save_img_folder, save_seg_folder])
    
    img = ff.nrrd_to_nii_orientation(np.rollaxis(img,2,0))
    seg = ff.nrrd_to_nii_orientation(np.rollaxis(seg,2,0))


    # borrow the affine and header
    f = nb.load(os.path.join(main_path, 'nii_img' , 'ID_0015', 'Org3D_frame1.nii.gz'))
    affine_matrix = f.affine
    header = f.header

    # borrowed x and y dim:
    header['dim'][1:3] = img.shape[1:3]
    # scale:
    scale = [spacing[0] / header['pixdim'][1],  spacing[1] / header['pixdim'][2],1]
    S = np.diag([scale[0], scale[1], scale[2], 1])
    
    new_affine = np.matmul(affine_matrix, S)
    affine_matrix = np.copy(new_affine)

    nb.save(nb.Nifti1Image(img,affine=affine_matrix, header = header), os.path.join(save_img_folder, 'Org3D_frame'+str(ed) +'.nii.gz'))
    nb.save(nb.Nifti1Image(seg, affine = affine_matrix, header = header), os.path.join(save_seg_folder, 'SAX_ED_seg.nii.gz'))




ID_0011 11
(0.78125, 0.78125, 8.0)
